### get the Visual Behavior dataset cache

In [1]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

C:\Users\marinag\Anaconda3\envs\visual_behavior_sdk\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
cache_dir = r'\\allen\programs\braintv\workgroups\nc-ophys\visual_behavior\platform_paper_cache'

cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=cache_dir)

### pick a random experiment and load the dataset 

In [3]:
experiments_table = cache.get_ophys_experiment_table()

In [4]:
ophys_experiment_id = experiments_table.index[0]

In [5]:
dataset = cache.get_behavior_ophys_experiment(ophys_experiment_id)

### get stimulus presentations and reward times

In [6]:
stimulus_presentations = dataset.stimulus_presentations.copy()
rewards = dataset.rewards.copy()

In [7]:
# stimulus_presentations has start and end times of each stimulus during the session
stimulus_presentations.head()

,duration,end_frame,image_index,image_name,image_set,index,omitted,start_frame,start_time,stop_time,is_change
stimulus_presentations_id,,,,,,,,,,,
0,0.25020,18001.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,0,False,17986,309.27537,309.52557,False
1,0.25021,18046.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,1,False,18031,310.02598,310.27619,False
2,0.25020,18091.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,2,False,18076,310.77660,311.02680,False
3,0.25019,18136.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,3,False,18121,311.52721,311.77740,False
4,0.25024,18181.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,4,False,18166,312.27782,312.52806,False


In [8]:
# rewards has the time of each reward that happened during the session 
rewards.head()

,volume,timestamps,autorewarded
0,0.005,322.91986,True
1,0.005,333.44514,True
2,0.005,364.23697,True
3,0.005,388.25660,True
4,0.005,455.04450,True


### method for associating reward times with stimulus presentation times

In [9]:
def rewards_each_flash(stimulus_presentations_df, rewards_df,
                       range_relative_to_stimulus_start=[0, 0.75]):
    '''
    Append a column to stimulus_presentations which contains the timestamps of rewards that occur
    in a range relative to the onset of the stimulus.

    Args:
        stimulus_presentations_df (pd.DataFrame): dataframe of stimulus presentations.
            Must contain: 'start_time'
        rewards_df (pd.DataFrame): rewards dataframe. Must contain 'timestamps'
        range_relative_to_stimulus_start (list with 2 elements): start and end of the range
            relative to the start of each stimulus to average the running speed.
    Returns:
        rewards_each_flash (pd.Series): reward times that fell within the window relative to each stim time
    '''

    reward_times = rewards_df['timestamps'].values
    rewards_each_flash = stimulus_presentations_df.apply(
        lambda row: reward_times[
            ((reward_times > row["start_time"] + range_relative_to_stimulus_start[0]) & 
             (reward_times < row["start_time"] + range_relative_to_stimulus_start[1]))],
        axis=1,)
    return rewards_each_flash


In [10]:
stimulus_presentations['rewards'] = rewards_each_flash(stimulus_presentations, rewards)

In [11]:
stimulus_presentations.head()

,duration,end_frame,image_index,image_name,image_set,index,omitted,start_frame,start_time,stop_time,is_change,rewards
stimulus_presentations_id,,,,,,,,,,,,
0,0.25020,18001.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,0,False,17986,309.27537,309.52557,False,[]
1,0.25021,18046.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,1,False,18031,310.02598,310.27619,False,[]
2,0.25020,18091.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,2,False,18076,310.77660,311.02680,False,[]
3,0.25019,18136.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,3,False,18121,311.52721,311.77740,False,[]
4,0.25024,18181.0,0,im065,Natural_Images_Lum_Matched_set_training_2017.0...,4,False,18166,312.27782,312.52806,False,[]


In [12]:
# look at values for rewards column
stimulus_presentations.rewards

stimulus_presentations_id
0       []
1       []
2       []
3       []
4       []
        ..
4796    []
4797    []
4798    []
4799    []
4800    []
Name: rewards, Length: 4801, dtype: object

In [13]:
# let's check the changes specifically
stimulus_presentations[stimulus_presentations.is_change==True].rewards

stimulus_presentations_id
18      [322.91986]
32      [333.44514]
73      [364.23697]
105      [388.2566]
194      [455.0445]
           ...     
4747             []
4757             []
4770             []
4784             []
4795             []
Name: rewards, Length: 186, dtype: object

In [14]:
# turn it in to a Boolean to get a True False for whether a stimulus was rewarded or not
stimulus_presentations['rewarded'] = [True if len(rewards)>0 else False for rewards in stimulus_presentations.rewards.values]

In [15]:
# let's check the reward times and rewarded Boolean column for the changes
stimulus_presentations[stimulus_presentations.is_change][['is_change', 'rewards', 'rewarded']]

,is_change,rewards,rewarded
stimulus_presentations_id,,,
18,True,[322.91986],True
32,True,[333.44514],True
73,True,[364.23697],True
105,True,[388.2566],True
194,True,[455.0445],True
...,...,...,...
4747,True,[],False
4757,True,[],False
4770,True,[],False


### Another way to get reward and behavioral trial information, including reward times

The `trials` attribute contains information just for image changes and sham changes (catch trials) and is useful for analysis of behavioral performance when you do not need information about each individual stimulus presentation in between the changes

In [16]:
trials = dataset.trials.copy()
trials.head()

,start_time,stop_time,lick_times,reward_time,reward_volume,hit,false_alarm,miss,stimulus_change,aborted,...,catch,auto_rewarded,correct_reject,trial_length,response_time,change_frame,change_time,response_latency,initial_image_name,change_image_name
trials_id,,,,,,,,,,,,,,,,,,,,,
0,309.25866,311.72736,[311.41045],NaN,0.000,False,False,False,False,True,...,False,False,False,2.46870,NaN,NaN,NaN,NaN,im065,im065
1,312.26114,314.16270,[313.84577],NaN,0.000,False,False,False,False,True,...,False,False,False,1.90156,NaN,NaN,NaN,NaN,im065,im065
2,314.51298,316.64807,[316.33114],NaN,0.000,False,False,False,False,True,...,False,False,False,2.13509,NaN,NaN,NaN,NaN,im065,im065
3,316.76480,319.61716,[319.30022],NaN,0.000,False,False,False,False,True,...,False,False,False,2.85236,NaN,NaN,NaN,NaN,im065,im065
4,319.76729,327.03990,"[323.22008, 323.38691, 323.75386, 323.90398, 3...",322.91986,0.005,False,False,False,True,False,...,False,True,False,7.27261,323.22008,18795.0,322.822945,0.397135,im065,im069


### Other useful functions

In [17]:

def licks_each_flash(stimulus_presentations_df, licks_df,
                     range_relative_to_stimulus_start=[0, 0.75]):
    '''
    Append a column to stimulus_presentations which contains the timestamps of licks that occur
    in a range relative to the onset of the stimulus.

    Args:
        stimulus_presentations_df (pd.DataFrame): dataframe of stimulus presentations.
            Must contain: 'start_time'
        licks_df (pd.DataFrame): lick dataframe. Must contain 'timestamps'
        range_relative_to_stimulus_start (list with 2 elements): start and end of the range
            relative to the start of each stimulus to average the running speed.
    Returns:
        licks_each_flash (pd.Series): lick times that fell within the window relative to each stim time
    '''

    lick_times = licks_df['timestamps'].values
    licks_each_flash = stimulus_presentations_df.apply(
        lambda row: lick_times[
            ((lick_times > row["start_time"] + range_relative_to_stimulus_start[0]) & 
             (lick_times < row["start_time"] + range_relative_to_stimulus_start[1]))],
            axis=1,)
    return licks_each_flash


def mean_running_speed(stimulus_presentations_df, running_speed_df,
                       range_relative_to_stimulus_start=[0, 0.25]):
    '''
    Append a column to stimulus_presentations which contains the mean running speed in a range relative to
    the stimulus start time.

    Args:
        stimulus_presentations_df (pd.DataFrame): dataframe of stimulus presentations.
            Must contain: 'start_time'
        running_speed_df (pd.DataFrame): dataframe of running speed.
            Must contain: 'speed', 'timestamps'
        range_relative_to_stimulus_start (list with 2 elements): start and end of the range
            relative to the start of each stimulus to average the running speed.
    Returns:
        flash_running_speed (pd.Series): mean running speed for each stimulus presentation.
    '''
    flash_running_speed = stimulus_presentations_df.apply(
        lambda row: trace_average(
            running_speed_df['speed'].values,
            running_speed_df['timestamps'].values,
            row["start_time"] + range_relative_to_stimulus_start[0],
            row["start_time"] + range_relative_to_stimulus_start[1],
        ),
        axis=1,
    )
    return flash_running_speed


def mean_pupil_area(stimulus_presentations_df, eye_tracking,
                    range_relative_to_stimulus_start=[0, 0.25]):
    '''
    Append a column to stimulus_presentations which contains the mean pupil area in a range relative to
    the stimulus start time.

    Args:
        stimulus_presentations_df (pd.DataFrame): dataframe of stimulus presentations.
            Must contain: 'start_time'
        eye_tracking (pd.DataFrame): dataframe of eye tracking data.
            Must contain: 'pupil_area', 'timestamps'
        range_relative_to_stimulus_start (list with 2 elements): start and end of the range
            relative to the start of each stimulus to average the pupil area.
    Returns:
        flash_running_speed (pd.Series): mean running speed for each stimulus presentation.
    '''
    flash_pupil_area = stimulus_presentations_df.apply(
        lambda row: trace_average(
            eye_tracking['pupil_area'].values,
            eye_tracking['timestamps'].values,
            row["start_time"] + range_relative_to_stimulus_start[0],
            row["start_time"] + range_relative_to_stimulus_start[1],
        ),
        axis=1,
    )
    return flash_pupil_area



